## 1. Setup and Imports

In [1]:
import os
import random
import pandas as pd
import sentencepiece as spm
import sacrebleu
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import zipfile
from tqdm import tqdm

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

PyTorch version: 2.9.1+cu128
CUDA available: True


## 2. Configuration and Hyperparameters

In [ ]:
# Data paths
TRAIN_SRC = "dataset/train/train.zh"
TRAIN_TGT = "dataset/train/train.vi"
TEST_SRC = "dataset/public_test/public_test.zh"
SAVE_DIR = "./checkpoints"
SPM_ZH_PREFIX = os.path.join(SAVE_DIR, "spm_zh")
SPM_VI_PREFIX = os.path.join(SAVE_DIR, "spm_vi")

# Model hyperparameters
VOCAB_SIZE = 5000
EMB_SIZE = 128
HID_SIZE = 128
BATCH_SIZE = 64
EPOCHS = 20
LR = 3e-5
MAX_LEN = 128
SEED = 42

# Device configuration
DEVICE = torch.device("cuda" )
print(f"Using device: {DEVICE}")

# Set random seeds for reproducibility
os.makedirs(SAVE_DIR, exist_ok=True)
random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)

Using device: cuda


## 3. Data Loading and Preprocessing

In [3]:
def read_lines(path):
    """Read lines from a text file."""
    with open(path, "r", encoding="utf-8") as f:
        return [l.strip() for l in f if l.strip()]

# Load training and test data
train_src = read_lines(TRAIN_SRC)
train_tgt = read_lines(TRAIN_TGT)
test_src = read_lines(TEST_SRC)

print(f"Training samples: {len(train_src)}")
print(f"Test samples: {len(test_src)}")
print(f"\nExample Chinese sentence: {train_src[0]}")
print(f"Example Vietnamese sentence: {train_tgt[0]}")

Training samples: 32061
Test samples: 1781

Example Chinese sentence: 我 会 给 您 拿 一些 。
Example Vietnamese sentence: Tôi sẽ mang cho bạn một_ít .


## 4. SentencePiece Tokenization

Train BPE tokenizers for both Chinese and Vietnamese.

In [4]:
def train_spm(input_file, model_prefix, vocab_size=VOCAB_SIZE):
    """Train a SentencePiece BPE model."""
    args = (
        f"--input={input_file} --model_prefix={model_prefix} --vocab_size={vocab_size} "
        "--model_type=bpe --character_coverage=1.0 "
        "--pad_id=0 --unk_id=1 --bos_id=2 --eos_id=3"
    )
    spm.SentencePieceTrainer.Train(args)
    print(f"Trained SentencePiece model: {model_prefix}.model")

def load_sp(model_path):
    """Load a trained SentencePiece model."""
    sp = spm.SentencePieceProcessor()
    sp.Load(model_path)
    return sp

In [5]:
# Train Chinese tokenizer
tmp_zh = os.path.join(SAVE_DIR, "tmp_zh.txt")
if not os.path.exists(SPM_ZH_PREFIX + ".model"):
    with open(tmp_zh, "w", encoding="utf-8") as f:
        for s in train_src:
            f.write(s + "\n")
    train_spm(tmp_zh, SPM_ZH_PREFIX)

# Train Vietnamese tokenizer
tmp_vi = os.path.join(SAVE_DIR, "tmp_vi.txt")
if not os.path.exists(SPM_VI_PREFIX + ".model"):
    with open(tmp_vi, "w", encoding="utf-8") as f:
        for s in train_tgt:
            f.write(s + "\n")
    train_spm(tmp_vi, SPM_VI_PREFIX)


from transformers import T5Tokenizer
# Load tokenizers
# sp_zh = load_sp(SPM_ZH_PREFIX + ".model")
# sp_vi = load_sp(SPM_VI_PREFIX + ".model")
sp_zh = T5Tokenizer(vocab_file="./checkpoints/spm_zh.model")
sp_vi = T5Tokenizer(vocab_file="./checkpoints/spm_vi.model")

print(f"\nChinese vocab size: {sp_zh.vocab_size}")
print(f"Vietnamese vocab size: {sp_vi.vocab_size}")

# Test tokenization
test_sent = train_src[0]
tokens = sp_zh.encode(test_sent)
print(f"\nExample tokenization:")
print(f"Original: {test_sent}")
print(f"Token IDs: {tokens[:20]}...")
print(f"Token was decoded: {sp_zh.decode(tokens[:20], skip_special_tokens=True)}")

/venv/olp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565



Chinese vocab size: 3000
Vietnamese vocab size: 3000

Example tokenization:
Original: 我 会 给 您 拿 一些 。
Token IDs: [5, 47, 28, 56, 109, 162, 4, 3]...
Token was decoded: 我 会 给 您 拿 一些 。


## 5. Dataset and DataLoader

In [6]:
class TranslationDataset(Dataset):
    """Dataset for Chinese-Vietnamese translation pairs."""
    
    def __init__(self, src, tgt, sp_src, sp_tgt, max_len=MAX_LEN):
        self.src = src
        self.tgt = tgt
        self.sp_src = sp_src
        self.sp_tgt = sp_tgt
        self.max_len = max_len

    def __len__(self):
        return len(self.src)

    def __getitem__(self, idx):
        # Add BOS (2) and EOS (3) tokens
        src_ids = [2] + self.sp_src.encode(self.src[idx])[:self.max_len-2] + [3]
        tgt_ids = self.sp_tgt.encode(self.tgt[idx])[:self.max_len-1] + [3]
        decoder_ids = [2] + self.sp_tgt.encode(self.tgt[idx])[:self.max_len-1]
        return torch.tensor(src_ids), torch.tensor(tgt_ids), torch.tensor(decoder_ids)

def collate_fn(batch):
    """Collate function to pad sequences to the same length."""
    srcs, tgts, decoders = zip(*batch)
    max_src = max(len(s) for s in srcs)
    max_tgt = max(len(t) for t in tgts)
    max_decoder = max(len(t) for t in decoders)
    
    
    # Pad with 0 (PAD token)
    src_pad = torch.zeros(len(batch), max_src, dtype=torch.long)
    tgt_pad = torch.zeros(len(batch), max_tgt, dtype=torch.long)
    decoder_pad = torch.zeros(len(batch), max_decoder, dtype=torch.long)

    for i, (s, t, d) in enumerate(zip(srcs, tgts, decoders)):
        src_pad[i, :len(s)] = s
        tgt_pad[i, :len(t)] = t
        decoder_pad[i, :len(d)] = d 

    return src_pad, tgt_pad, decoder_pad

In [7]:
# Create training dataset and dataloader
# Split data: 90% train, 10% validation
total_samples = len(train_src)
train_size = int(0.9 * total_samples)

train_src_split = train_src[:train_size]
train_tgt_split = train_tgt[:train_size]
valid_src = train_src[train_size:]
valid_tgt = train_tgt[train_size:]

print(f"Total samples: {total_samples}")
print(f"Training samples: {len(train_src_split)}")
print(f"Validation samples: {len(valid_src)}")

dataset = TranslationDataset(train_src_split, train_tgt_split, sp_zh, sp_vi)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn, num_workers=4, pin_memory=True)

valid_dataset = TranslationDataset(valid_src, valid_tgt, sp_zh, sp_vi)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn, num_workers=2, pin_memory=True)

print(f"Training batches: {len(dataloader)}")
print(f"Validation batches: {len(valid_loader)}")

Total samples: 32061
Training samples: 28854
Validation samples: 3207
Training batches: 451
Validation batches: 101


In [8]:
for sample in valid_loader: 
    print(sample)
    break

[tensor([[   2,    5,   83,    7,  222,  889,  681,   13,  222,  973,  981,   43,
           68,  506,    4,    3,    3,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0],
        [   2,    5,   83,  222,  595,  491,   43,  222,  533,  290,    4,    3,
            3,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0],
        [   2,    8,   36,  247,  222,  560,  295,  693,   74,  426,  222, 1270,
          309,    7,  222,  465,  345,   14,    9,    6,    3,    3,    0,    0,
            0,    0,    0,    0,    0,    0,    0],
        [   2,    8,   17,  222,  505,  798,  222,  451,  430,  222,  585,  381,
            9,    6,    3,    3,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0],
        [   2,   12,   13,   11,   19,  222,  619,  399,  219,  735,  222,  504,
          577,  222,  419,  793,    7,   23,  220,  266,    4,

## 6. Model Architecture

### T5 model

In [ ]:
from transformers import T5ForConditionalGeneration, T5Config
class T5model (nn.Module): 
    def __init__(self, vocab_size):
        super().__init__()
        config = T5Config(
            vocab_size=vocab_size, 
            pad_token_id=0,
            eos_token_id=3, 
            bos_token_id=2
        )
        self.model = T5ForConditionalGeneration(config=config)
    def forward(self, encoder_input, decoder_input, label): 
        logits = self.model(encoder_input, decoder_input_ids=decoder_input, labels=label)
        return logits.logits
    
    def generate(self, src): 
        return self.model.generate(src)
    
model = T5model(sp_vi.vocab_size).to(DEVICE)
print(model.eval())

2025-12-07 02:25:07.380535: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-07 02:25:07.409697: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-07 02:25:08.066174: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


T5model(
  (model): T5ForConditionalGeneration(
    (shared): Embedding(3000, 512)
    (encoder): T5Stack(
      (embed_tokens): Embedding(3000, 512)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=512, out_features=512, bias=False)
                (k): Linear(in_features=512, out_features=512, bias=False)
                (v): Linear(in_features=512, out_features=512, bias=False)
                (o): Linear(in_features=512, out_features=512, bias=False)
                (relative_attention_bias): Embedding(32, 8)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseActDense(
                (wi): Linear(in_features=512, out_features=2048, bias=False)
                (wo): Linear(in_features=2048,

### Encoder-Decoder with GRU

In [10]:
# class EncoderRNN(nn.Module):
#     """GRU-based encoder."""
    
#     def __init__(self, vocab_size, emb_size, hidden_size):
#         super().__init__()
#         self.embedding = nn.Embedding(vocab_size, emb_size, padding_idx=0)
#         self.dropout = nn.Dropout(0.3)
#         self.rnn = nn.GRU(emb_size, hidden_size, batch_first=True)

#     def forward(self, src):
#         emb = self.dropout(self.embedding(src))
#         _, hidden = self.rnn(emb)
#         return hidden


# class DecoderRNN(nn.Module):
#     """GRU-based decoder with teacher forcing."""
    
#     def __init__(self, vocab_size, emb_size, hidden_size):
#         super().__init__()
#         self.embedding = nn.Embedding(vocab_size, emb_size, padding_idx=0)
#         self.dropout = nn.Dropout(0.3)
#         self.rnn = nn.GRU(emb_size, hidden_size, batch_first=True)
#         self.fc = nn.Linear(hidden_size, vocab_size)

#     def forward(self, input_step, hidden):
#         emb = self.dropout(self.embedding(input_step))
#         output, hidden = self.rnn(emb, hidden)
#         pred = self.fc(output.squeeze(1))
#         return pred, hidden


# class Seq2Seq(nn.Module):
#     """Sequence-to-sequence model."""
    
#     def __init__(self, encoder, decoder):
#         super().__init__()
#         self.encoder = encoder
#         self.decoder = decoder

#     def forward(self, src, tgt, teacher_forcing_ratio=0.3):
#         batch_size = src.size(0)
#         tgt_len = tgt.size(1)
#         vocab_size = self.decoder.fc.out_features
        
#         # Store outputs
#         outputs = torch.zeros(batch_size, tgt_len, vocab_size).to(src.device)
        
#         # Encode source sentence
#         hidden = self.encoder(src)
        
#         # Start with BOS token
#         input_step = tgt[:, 0].unsqueeze(1)
        
#         # Decode step by step
#         for t in range(1, tgt_len):
#             pred, hidden = self.decoder(input_step, hidden)
#             outputs[:, t] = pred
            
#             # Teacher forcing
#             teacher_force = random.random() < teacher_forcing_ratio
#             input_step = tgt[:, t].unsqueeze(1) if teacher_force else pred.argmax(1).unsqueeze(1)
        
#         return outputs

In [11]:
# # Initialize model
# encoder = EncoderRNN(sp_zh.GetPieceSize(), EMB_SIZE, HID_SIZE)
# decoder = DecoderRNN(sp_vi.GetPieceSize(), EMB_SIZE, HID_SIZE)
# model = Seq2Seq(encoder, decoder).to(DEVICE)

# # Count parameters
# def count_parameters(model):
#     return sum(p.numel() for p in model.parameters() if p.requires_grad)

# print(f"Model has {count_parameters(model):,} trainable parameters")
# print(f"\nModel architecture:")
# print(model)

## 7. Training Functions

In [12]:
def train_epoch(model, dataloader, criterion, optimizer):
    """Train for one epoch."""
    model.train()
    total_loss = 0
    
    for src, tgt, dcd in dataloader:
        src, tgt, dcd = src.to(DEVICE), tgt.to(DEVICE), dcd.to(DEVICE)
        
        optimizer.zero_grad()
        output = model(src, dcd, tgt)
        
        # Calculate loss (ignore first BOS token)
        loss = criterion(
            output[:, 1:].reshape(-1, output.size(-1)), 
            tgt[:, 1:].reshape(-1)
        )
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        
        total_loss += loss.item()

    return total_loss / len(dataloader)


@torch.no_grad()
def evaluate_bleu(model, dataloader, sp_tgt):
    """Evaluate model using SacreBLEU metric."""
    model.eval()

    pred_texts = []
    target_texts = []
    
    pbar = tqdm(dataloader, desc="Evaluating", leave=False)
    for src, tgt, dcd in pbar: 
        pred_id = model.generate(src.to(DEVICE))

        target_id = tgt # tgt_encoding_4_encoder [w1, w2, w3, EOS]


        for pred, target in zip(pred_id, target_id):
            pred_list = pred.tolist()  # [w1, w2, w3, EOS, PAD, PAD, PAD,...]
            target_list = target.tolist()  # [SOS, w1, w2, w3, EOS, PAD, PAD, PAD,...]

            target_texts.append(
                sp_tgt.decode(target_list, skip_special_tokens=True)
            )
            pred_texts.append(
                sp_tgt.decode(pred_list, skip_special_tokens=True)
            )

    bleu = sacrebleu.corpus_bleu(pred_texts, [[ref] for ref in target_texts])
    return bleu.score

## 8. Training Loop

In [13]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss(ignore_index=0)  # Ignore padding
optimizer = optim.Adam(model.parameters(), lr=LR)

print("Starting training...\n")

# Training loop with best model saving
best_bleu = 0.0
best_model_path = os.path.join(SAVE_DIR, "best_model.pt")

for epoch in range(1, EPOCHS + 1):
    loss = train_epoch(model, dataloader, criterion, optimizer)
    bleu = evaluate_bleu(model, valid_loader, sp_vi)
    
    # Save best model
    if bleu > best_bleu:
        best_bleu = bleu
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'bleu': bleu,
            'loss': loss
        }, best_model_path)
        print(f"Epoch {epoch:02d} | Loss={loss:.3f} | SacreBLEU={bleu:.2f} Best model saved!")
    else:
        print(f"Epoch {epoch:02d} | Loss={loss:.3f} | SacreBLEU={bleu:.2f}")

print(f"\nTraining completed!")
print(f"Best validation BLEU: {best_bleu:.2f}")
print(f"Best model saved to: {best_model_path}")

# Load best model for inference
checkpoint = torch.load(best_model_path)
model.load_state_dict(checkpoint['model_state_dict'])
print(f"Loaded best model from epoch {checkpoint['epoch']}")

Starting training...



Epoch 01 | Loss=4.447 | SacreBLEU=58.74 Best model saved!


Epoch 02 | Loss=3.644 | SacreBLEU=27.65


Epoch 03 | Loss=3.249 | SacreBLEU=17.11


Epoch 04 | Loss=2.966 | SacreBLEU=47.59


...

Epoch 16 | Loss=1.440 | SacreBLEU=0.00


Epoch 17 | Loss=1.366 | SacreBLEU=0.00


Epoch 18 | Loss=1.293 | SacreBLEU=0.00


Epoch 19 | Loss=1.226 | SacreBLEU=0.00


Epoch 20 | Loss=1.161 | SacreBLEU=0.00

Training completed!
Best validation BLEU: 58.74
Best model saved to: ./checkpoints/best_model.pt
Loaded best model from epoch 1


## 9. Translation on Test Set

In [ ]:
@torch.no_grad()
def translate_test(model, test_src, sp_src, sp_tgt, out_path):
    """Translate test set and save to CSV."""
    model.eval()
    outputs = []
    
    for s in tqdm(test_src, desc="Translating"):
        # Tokenize source sentence
        src_ids = [2] + sp_src.encode(s)[:MAX_LEN-2] + [3]
        src_tensor = torch.tensor(src_ids, dtype=torch.long, device=DEVICE).unsqueeze(0)
        

        logits = model.generate(src_tensor).tolist()[0]
        # Decode to Vietnamese text
        vi_sent = sp_tgt.decode(logits)
        outputs.append(vi_sent)
    
    # Save to CSV
    df = pd.DataFrame({"tieng_trung": test_src, "tieng_viet": outputs})
    df.to_csv(out_path, index=False, encoding="utf-8-sig")
    
    return out_path

In [ ]:
# Translate test set
submission_path = os.path.join("public_submission.csv")
translate_test(model, test_src, sp_zh, sp_vi, submission_path)

print(f"Translation completed!")
print(f"Saved to: {submission_path}")

# Display sample translations
df_result = pd.read_csv(submission_path)
print("\nSample translations:")
print(df_result.head(10))

Translating:   0%|          | 8/1781 [00:00<00:47, 36.96it/s]

[2, 2829, 78, 49, 25, 182, 29, 25, 182, 182, 182, 182, 182, 182, 8, 3]
[2, 2829, 78, 49, 38, 27, 2829, 57, 49, 38, 47, 2829, 368, 96, 15, 3]
[2, 27, 2829, 57, 49, 25, 47, 2829, 325, 92, 2829, 441, 22, 15, 3]
[2, 2829, 78, 25, 91, 74, 102, 8, 3]
[2, 25, 79, 47, 124, 102, 8, 3]
[2, 2829, 78, 49, 25, 47, 2829, 368, 47, 2829, 368, 8, 3]
[2, 2829, 466, 466, 466, 15, 3]
[2, 79, 47, 124, 62, 8, 3]
[2, 2829, 78, 74, 74, 8, 3]


Translating:   1%|          | 16/1781 [00:00<00:50, 35.08it/s]

[2, 2829, 401, 29, 38, 27, 2829, 57, 103, 2829, 511, 511, 511, 511, 22, 15, 3]
[2, 2829, 78, 49, 25, 47, 2829, 368, 38, 22, 15, 3]
[2, 2829, 78, 49, 47, 124, 62, 29, 27, 2829, 57, 8, 3]
[2, 2829, 78, 49, 38, 36, 136, 51, 38, 36, 101, 15, 3]
[2, 2829, 78, 38, 27, 2829, 57, 49, 38, 47, 2829, 368, 136, 8, 3]
[2, 2829, 78, 49, 47, 124, 62, 22, 15, 3]
[2, 2829, 78, 49, 25, 53, 74, 102, 8, 3]
[2, 138, 2829, 205, 198, 15, 3]


Translating:   1%|▏         | 24/1781 [00:00<00:52, 33.78it/s]

...

[2, 25, 114, 214, 214, 214, 214, 214, 214, 214, 214, 214, 214, 214, 214, 214, 214, 214, 214, 214, 214]
[2, 36, 47, 2829, 368, 51, 25, 8, 3]
[2, 2829, 78, 38, 27, 2829, 57, 8, 3]
[2, 2829, 78, 38, 27, 2829, 57, 103, 22, 15, 3]
[2, 2829, 78, 29, 116, 2829, 123, 22, 2829, 57, 8, 3]
[2, 2829, 165, 29, 38, 22, 182, 182, 38, 165, 165, 165, 165, 165, 165, 165, 165, 165, 165, 165, 165]
[2, 2829, 401, 29, 25, 22, 182, 8, 3]


Translating:  76%|███████▋  | 1362/1781 [00:43<00:14, 28.28it/s]

[2, 2829, 78, 25, 27, 2829, 57, 8, 3]
[2, 2829, 78, 49, 25, 47, 2829, 368, 124, 62, 71, 22, 15, 3]
[2, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38]
[2, 2829, 78, 38, 59, 135, 15, 3]
[2, 36, 136, 136, 15, 3]
[2, 2829, 78, 49, 25, 182, 8, 3]
[2, 2829, 78, 38, 36, 38, 51, 38, 36, 101, 15, 3]


Translating:  77%|███████▋  | 1371/1781 [00:43<00:13, 29.78it/s]

[2, 2829, 78, 49, 47, 2829, 368, 51, 38, 8, 3]
[2, 2829, 78, 49, 25, 8, 3]
[2, 2829, 78, 38, 29, 38, 27, 2829, 57, 103, 2829, 78, 49, 38, 22, 15, 3]
[2, 2829, 78, 49, 25, 47, 2829, 325, 325, 325, 325, 325, 325, 325, 325, 22, 15, 3]
[2, 2829, 78, 49, 47, 124, 62, 59, 135, 15, 3]
[2, 2829, 78, 49, 38, 74, 47, 2829, 368, 136, 51, 38, 8, 3]


## 10. Create Submission ZIP

In [ ]:
# Create ZIP file for submission
zip_path = os.path.join("public_submission.zip")
with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipf.write(submission_path, arcname="public_submission.csv")

print(f"Submission ZIP created: {zip_path}")
print(f"File size: {os.path.getsize(zip_path) / 1024:.2f} KB")